<a href="https://colab.research.google.com/github/abrahamkoloboe27/python-and-gen-ai-data-science-growup-ai/blob/main/s11_rag_demo/s11_rag_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# S11 — RAG (Retrieval-Augmented Generation) : Cours Complet & Implémentation

## 🎯 Objectifs
- Comprendre ce qu'est le RAG et pourquoi l'utiliser
- Maîtriser les différents composants d'un pipeline RAG
- Connaître les différents types de RAG (Naive, Advanced, Modular/Agentic)
- Comprendre la gestion de mémoire dans les systèmes RAG
- Implémenter un RAG complet **sans LangChain** (OpenAI + ChromaDB + PyMuPDF)
- Implémenter le même RAG **avec LangChain** pour comparer les deux approches

## 📋 Contenu
1. Introduction au RAG : définition, motivation, architecture
2. Les composants du RAG
3. Les types de RAG
4. Gestion de la mémoire
5. Installation et configuration
6. Implémentation RAG sans LangChain
7. Implémentation RAG avec LangChain
8. Comparaison et bonnes pratiques

---
# 🧠 Section 1 — Introduction au RAG

---

## 1.1 Qu'est-ce que le RAG ?

**RAG (Retrieval-Augmented Generation)** est une architecture qui améliore les LLMs en leur donnant accès à des sources de connaissances **externes et actualisées** au moment de la génération.

Au lieu de répondre uniquement avec sa mémoire interne (les poids du modèle), un système RAG :

1. **Récupère** des documents pertinents depuis une base de connaissances
2. **Augmente** le prompt de l'utilisateur avec ces documents
3. **Génère** une réponse fondée sur ces documents

### Analogie 📚

Imaginez un étudiant qui passe un examen :
- **LLM seul** = étudiant qui répond de mémoire (peut halluciner)
- **RAG** = étudiant qui a accès à ses notes et peut les consulter avant de répondre

---

## 1.2 Pourquoi le RAG ?

### Les limites des LLMs sans RAG

| Problème | Description | Impact |
|----------|-------------|--------|
| **Hallucinations** | Le LLM invente des faits | Réponses incorrectes |
| **Knowledge cutoff** | Connaissance figée à la date d'entraînement | Info obsolète |
| **Données privées** | Le modèle ne connaît pas vos données internes | Pas d'utilisation en entreprise |
| **Manque de traçabilité** | On ne sait pas d'où vient la réponse | Difficile à auditer |
| **Coût de fine-tuning** | Re-entraîner pour chaque nouvelle connaissance | Très cher |

### Les avantages du RAG

✅ **Réponses vérifiables** : chaque affirmation est ancrée dans un document source  
✅ **Connaissances actualisées** : il suffit de mettre à jour la base de données  
✅ **Données privées** : fonctionne avec vos propres documents  
✅ **Coût réduit** : pas de re-entraînement nécessaire  
✅ **Moins d'hallucinations** : le LLM s'appuie sur des faits fournis  

---

## 1.3 Architecture générale du RAG

```
┌─────────────────────────────────────────────────────────────────┐
│                     PHASE D'INDEXATION                          │
│                                                                 │
│  Documents  →  Chunking  →  Embeddings  →  Vector Store        │
│  (PDF, etc)     (split)     (encode)       (ChromaDB, FAISS)   │
└─────────────────────────────────────────────────────────────────┘

┌─────────────────────────────────────────────────────────────────┐
│                     PHASE DE REQUÊTE                            │
│                                                                 │
│  Question  →  Embedding  →  Retrieval  →  Augmentation  →  LLM │
│  (user)       (encode)      (top-k)      (prompt+context)       │
│                                                      │         │
│                                                      ▼         │
│                                                  Réponse        │
└─────────────────────────────────────────────────────────────────┘
```

### Flux de données

```
          INDEXATION                       REQUÊTE
         ──────────                       ───────
 PDF ──► Chunks ──► Embeddings          Question
                        │                   │
                        ▼                   ▼
                   ChromaDB ◄──── Embedding de la question
                        │
                        │  (similarité cosinus)
                        ▼
                   Top-K chunks
                        │
                        ▼
                   Prompt enrichi ──► GPT-4 ──► Réponse
```

---
# 🔧 Section 2 — Les Composants du RAG

---

## 2.1 Document Loader (Chargeur de documents)

Le point d'entrée du pipeline. Il charge et normalise les données sources.

**Sources supportées :**
- 📄 **PDF** : PyMuPDF (fitz), pypdf, PDFPlumber
- 📝 **Word/Excel** : python-docx, openpyxl
- 🌐 **Web** : BeautifulSoup, Playwright
- 🗃️ **Base de données** : SQLAlchemy, connecteurs DB
- 📧 **Emails/Slack** : connecteurs spécifiques

---

## 2.2 Text Splitter (Découpage en chunks)

Un LLM a une **fenêtre de contexte limitée** (ex: 8k, 128k tokens). On ne peut pas injecter des centaines de pages. Il faut donc découper les documents en **chunks** (morceaux).

### Stratégies de chunking

| Stratégie | Description | Avantages | Inconvénients |
|-----------|-------------|-----------|---------------|
| **Fixed size** | Chunks de N caractères | Simple, uniforme | Coupe parfois au milieu d'une idée |
| **Overlap** | Chevauchement entre chunks | Préserve le contexte aux jointures | Redondance |
| **Sentence** | Découpe par phrase | Préserve le sens | Chunks de taille variable |
| **Recursive** | Essaie `\n\n`, `\n`, ` ` | Intelligent, adaptable | Plus complexe |
| **Semantic** | Regroupement sémantique | Meilleure cohérence | Coûteux en calcul |

### Paramètres clés
- **chunk_size** : taille d'un chunk (ex: 500 tokens)
- **chunk_overlap** : chevauchement entre chunks (ex: 50 tokens)

```
Document: [...paragraphe 1...][...paragraphe 2...][...paragraphe 3...]

Chunk 1: [──────500 tokens──────]
Chunk 2:               [──────500 tokens──────]
                   ←50→ (overlap)
Chunk 3:                              [──────500 tokens──────]
```

---

## 2.3 Embedding Model (Modèle d'embeddings)

Transforme chaque chunk en **vecteur dense** (représentation numérique) qui capture son sens sémantique.

### Modèles populaires

| Modèle | Dimension | Coût | Performance |
|--------|-----------|------|-------------|
| `text-embedding-3-small` (OpenAI) | 1536 | Payant | ⭐⭐⭐⭐ |
| `text-embedding-3-large` (OpenAI) | 3072 | Payant | ⭐⭐⭐⭐⭐ |
| `all-MiniLM-L6-v2` (Sentence-Transformers) | 384 | Gratuit | ⭐⭐⭐ |
| `paraphrase-multilingual-MiniLM-L12-v2` | 384 | Gratuit | ⭐⭐⭐ (multi-langue) |
| `nomic-embed-text` (Ollama local) | 768 | Gratuit | ⭐⭐⭐⭐ |

---

## 2.4 Vector Store (Base de données vectorielle)

Stocke les embeddings et permet la **recherche par similarité** (nearest neighbor search).

### Solutions disponibles

| Solution | Type | Cas d'usage |
|----------|------|-------------|
| **ChromaDB** | Local/Cloud | Développement, MVP |
| **FAISS** | Local (in-memory) | Recherche rapide, pas de persistance |
| **Pinecone** | Cloud managed | Production à grande échelle |
| **Weaviate** | Open-source | Entreprise, GraphQL |
| **Qdrant** | Open-source | Haute performance, filtrage avancé |
| **Milvus** | Open-source | Milliards de vecteurs |

> Dans ce notebook, nous utilisons **ChromaDB** : léger, local, aucune configuration serveur requise.

---

## 2.5 Retriever (Récupérateur)

Prend la question de l'utilisateur, la transforme en embedding, et retrouve les **K chunks les plus similaires** dans le vector store.

### Types de recherche

- **Dense retrieval** : basé sur la similarité d'embeddings (sémantique)
- **Sparse retrieval** : BM25/TF-IDF (mots-clés exacts)
- **Hybrid retrieval** : combinaison des deux (meilleur des deux mondes)

---

## 2.6 LLM / Generator (Générateur)

Reçoit le prompt enrichi (question + contexte récupéré) et génère la réponse finale.

### Prompt template typique

```
Tu es un assistant expert. Réponds à la question en te basant UNIQUEMENT 
sur le contexte fourni. Si l'information n'est pas dans le contexte, 
dis-le clairement.

Contexte:
{chunk_1}
{chunk_2}
{chunk_3}

Question: {question}

Réponse:
```

---
# 🗂️ Section 3 — Les Types de RAG

---

## 3.1 Naive RAG (RAG Simple)

L'approche de base, décrite dans le papier original de Lewis et al. (2020).

```
Question ──► Embedding ──► Vector Search ──► Prompt ──► LLM ──► Réponse
```

**✅ Avantages :** Simple à implémenter, efficace pour des cas simples  
**❌ Inconvénients :** Peut manquer de contexte, pas de raffinement de la query, peu robuste aux questions complexes

---

## 3.2 Advanced RAG

Améliore le Naive RAG à chaque étape du pipeline.

### Pre-retrieval (avant la recherche)

| Technique | Description |
|-----------|-------------|
| **Query rewriting** | Reformuler la question pour améliorer le retrieval |
| **HyDE** (Hypothetical Document Embeddings) | Générer un document hypothétique, puis l'embedder |
| **Step-back prompting** | Poser une question plus générale d'abord |
| **Multi-query** | Générer plusieurs variantes de la question |

### Post-retrieval (après la recherche)

| Technique | Description |
|-----------|-------------|
| **Reranking** | Re-classer les chunks par un modèle cross-encoder |
| **Context compression** | Résumer/filtrer les chunks récupérés |
| **Lost in the middle** | Placer les chunks importants au début/fin du contexte |

### Exemple : Query Rewriting
```
Question originale : "C'est quoi GPT ?"
           ↓ (LLM rewriting)
Questions reformulées :
  1. "Qu'est-ce que GPT (Generative Pre-trained Transformer) ?"
  2. "Quel est le fonctionnement du modèle GPT d'OpenAI ?"
  3. "Quelle est l'architecture du modèle de langage GPT ?"
           ↓ (retrieve for each, merge results)
Contexte enrichi ──► LLM ──► Réponse
```

---

## 3.3 Modular RAG (RAG Modulaire / Agentic)

L'approche la plus avancée. Le système peut **décider dynamiquement** quels modules utiliser.

```
                         ┌─────────────────────┐
                         │    Orchestrateur    │
                         │     (LLM Agent)     │
                         └──────────┬──────────┘
                                    │
               ┌────────────────────┼────────────────────┐
               ▼                    ▼                    ▼
    ┌──────────────────┐  ┌──────────────────┐  ┌──────────────────┐
    │  Vector Search   │  │   Web Search     │  │  SQL Database    │
    │  (documents)     │  │  (temps réel)    │  │  (données struct)│
    └──────────────────┘  └──────────────────┘  └──────────────────┘
```

### Sous-types

| Type | Description | Usage |
|------|-------------|-------|
| **RAG-Fusion** | Fusionne résultats de plusieurs retrievers | Précision maximale |
| **Self-RAG** | Le LLM décide lui-même quand récupérer | Flexibilité |
| **CRAG** (Corrective RAG) | Évalue la pertinence et corrige si nécessaire | Robustesse |
| **Iterative RAG** | Plusieurs cycles de retrieval | Questions complexes |
| **Graph RAG** | Utilise un graphe de connaissances | Relations complexes |

---

## 3.4 Quand utiliser quel type ?

```
Complexité de la question
         ▲
         │
  Élevée │ ◄── Modular/Agentic RAG
         │         (questions multi-étapes, sources multiples)
  Moyenne│ ◄── Advanced RAG
         │         (questions nécessitant reformulation)
  Faible │ ◄── Naive RAG
         │         (Q&A simple sur documents)
         └─────────────────────────────────────────► Maturité du projet
              MVP          Prod          Entreprise
```

---
# 💾 Section 4 — Gestion de la Mémoire

---

## 4.1 Pourquoi la gestion de mémoire est-elle importante ?

Un LLM a une **fenêtre de contexte** (context window) limitée. Si vous essayez d'injecter trop de texte, ça dépasse la limite et génère une erreur. Il faut donc gérer intelligemment ce qu'on met dans le prompt.

| Modèle | Context Window |
|--------|---------------|
| GPT-3.5-turbo | 16k tokens |
| GPT-4 | 128k tokens |
| GPT-4o | 128k tokens |
| Claude 3.5 Sonnet | 200k tokens |
| Gemini 1.5 Pro | 1M tokens |

> 💡 **Règle de base** : 1 token ≈ 4 caractères ≈ 0.75 mot en anglais

---

## 4.2 Types de mémoire dans un système RAG

### Mémoire à court terme (In-context memory)
Ce qui est dans le prompt courant. Limité par la context window.
```
[System prompt] + [Historique conversation] + [Contexte RAG] + [Question] → LLM
```

### Mémoire à long terme (External memory)
Stockée en dehors du LLM :
- **Vector store** : retrouvée par similarité sémantique
- **Base de données** : retrouvée par ID ou requête SQL
- **Cache** : résultats précalculés pour des queries fréquentes

### Mémoire de conversation
Historique des échanges précédents. Plusieurs stratégies :

| Stratégie | Description | Avantages | Inconvénients |
|-----------|-------------|-----------|---------------|
| **Buffer** | Garder tous les messages | Complet | Dépasse vite la context window |
| **Window** | Garder les N derniers messages | Équilibré | Perd le contexte ancien |
| **Summary** | Résumer les anciens messages | Compact | Perd des détails |
| **Vector memory** | Stocker et retriever les échanges | Scalable | Complexe |

---

## 4.3 Stratégies de gestion du contexte RAG

```python
# Budget de tokens (exemple GPT-3.5-turbo = 16k tokens)
TOTAL_BUDGET = 16_000
SYSTEM_PROMPT_TOKENS = 200
CONVERSATION_HISTORY_TOKENS = 2_000
ANSWER_RESERVE_TOKENS = 1_000

# Budget disponible pour le contexte RAG
RAG_CONTEXT_BUDGET = TOTAL_BUDGET - SYSTEM_PROMPT_TOKENS - CONVERSATION_HISTORY_TOKENS - ANSWER_RESERVE_TOKENS
# = 12_800 tokens pour les chunks récupérés
```

### Lost in the Middle Problem
Les LLMs ont tendance à mieux utiliser l'information placée **au début** ou **à la fin** du contexte. Le milieu est souvent "perdu".

**Solution :** Placer les chunks les plus pertinents aux extrémités du contexte :
```
Contexte = [Chunk le plus pertinent] + [Chunks moyens] + [2e plus pertinent]
```

---

## 4.4 Conversational RAG (RAG avec historique)

Pour maintenir une conversation, il faut reformuler la question en tenant compte de l'historique :

```
Tour 1:
  User: "Qu'est-ce que le machine learning ?"
  Bot: "Le ML est..."

Tour 2:
  User: "Et comment ça s'applique à la médecine ?"
  
  ← Sans reformulation, "ça" est ambigu pour le retriever !
  
  Reformulation automatique (LLM) :
  "Comment le machine learning s'applique-t-il à la médecine ?"
  
  ← Maintenant le retriever peut trouver les bons chunks
```

---
# ⚙️ Section 5 — Installation et Configuration

---

In [ ]:
# Installation des dépendances
!pip install -q openai chromadb pymupdf python-dotenv tiktoken

In [ ]:
# Installation des dépendances LangChain (pour la Section 7)
!pip install -q langchain langchain-openai langchain-chroma langchain-community langchain-text-splitters

In [ ]:
import os
from dotenv import load_dotenv

# Charger les variables d'environnement depuis .env
load_dotenv()

# Clé API OpenAI - à définir dans votre .env ou directement ici (déconseillé en prod)
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

if not OPENAI_API_KEY:
    # Pour Google Colab : utiliser les secrets
    try:
        from google.colab import userdata
        OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
        os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY
        print("✅ Clé OpenAI chargée depuis les secrets Colab")
    except Exception:
        print("⚠️ OPENAI_API_KEY non trouvée. Définissez-la dans votre .env ou dans les secrets Colab.")
        print("Pour tester sans clé API, les cellules de génération seront simulées.")
else:
    print("✅ Clé OpenAI chargée depuis .env")

In [ ]:
# Créer un document PDF de démonstration
# (si vous avez déjà un PDF, adaptez le chemin ci-dessous)

import os

# Vérifier si un PDF existe déjà dans data/
DATA_DIR = "data"
PDF_PATH = os.path.join(DATA_DIR, "document_demo.pdf")

os.makedirs(DATA_DIR, exist_ok=True)

# Créer un PDF de démonstration avec reportlab si aucun PDF n'est disponible
if not os.path.exists(PDF_PATH):
    try:
        from reportlab.lib.pagesizes import A4
        from reportlab.lib.styles import getSampleStyleSheet
        from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer
        from reportlab.lib.units import cm
    except ImportError:
        import subprocess
        subprocess.run(["pip", "install", "-q", "reportlab"], check=True)
        from reportlab.lib.pagesizes import A4
        from reportlab.lib.styles import getSampleStyleSheet
        from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer
        from reportlab.lib.units import cm

    doc = SimpleDocTemplate(PDF_PATH, pagesize=A4)
    styles = getSampleStyleSheet()
    story = []

    content = [
        ("h1", "Introduction à l'Intelligence Artificielle"),
        ("body", "L'intelligence artificielle (IA) est un domaine de l'informatique qui vise à créer des systèmes capables d'effectuer des tâches qui nécessitent normalement l'intelligence humaine. Ces tâches incluent la reconnaissance vocale, la prise de décision, la traduction entre langues, et la perception visuelle."),
        ("h2", "1. Le Machine Learning"),
        ("body", "Le machine learning (apprentissage automatique) est une sous-discipline de l'IA qui permet aux systèmes d'apprendre à partir des données sans être explicitement programmés. Il existe trois grands types : l'apprentissage supervisé, non supervisé et par renforcement."),
        ("body", "L'apprentissage supervisé utilise des données étiquetées pour entraîner un modèle. Par exemple, un modèle de classification d'images apprend à reconnaître des chats en voyant des milliers d'images annotées 'chat' ou 'pas chat'."),
        ("body", "L'apprentissage non supervisé découvre des structures cachées dans des données non étiquetées. Le clustering K-means est un exemple classique qui regroupe des points de données similaires ensemble."),
        ("body", "L'apprentissage par renforcement entraîne un agent à prendre des décisions en maximisant une récompense cumulative. AlphaGo de DeepMind, qui a battu le champion du monde de Go, utilise cette technique."),
        ("h2", "2. Le Deep Learning"),
        ("body", "Le deep learning est une branche du machine learning basée sur des réseaux de neurones artificiels à plusieurs couches (d'où 'deep'). Ces réseaux peuvent apprendre des représentations hiérarchiques des données."),
        ("body", "Les réseaux de neurones convolutifs (CNN) sont particulièrement efficaces pour le traitement d'images. Ils utilisent des filtres pour détecter des caractéristiques comme les bords, les textures, puis des formes plus complexes."),
        ("body", "Les transformers, introduits en 2017 dans le papier 'Attention Is All You Need', ont révolutionné le traitement du langage naturel. Ils utilisent un mécanisme d'attention pour capturer les relations entre mots dans une séquence."),
        ("h2", "3. Les Grands Modèles de Langage (LLM)"),
        ("body", "Les LLMs (Large Language Models) sont des modèles de deep learning entraînés sur de vastes corpus de texte. GPT-4 d'OpenAI, Claude d'Anthropic, et Gemini de Google sont des exemples de LLMs de pointe."),
        ("body", "Ces modèles sont capables de générer du texte cohérent, de répondre à des questions, de traduire, de résumer des documents, et même d'écrire du code. Ils sont basés sur l'architecture Transformer et contiennent des milliards de paramètres."),
        ("body", "Le fine-tuning permet d'adapter un LLM pré-entraîné à une tâche spécifique avec un dataset plus petit. Le RLHF (Reinforcement Learning from Human Feedback) est utilisé pour aligner les LLMs avec les préférences humaines."),
        ("h2", "4. Le RAG (Retrieval-Augmented Generation)"),
        ("body", "Le RAG est une technique qui améliore les LLMs en leur donnant accès à des sources de connaissances externes. Au lieu de répondre uniquement depuis leur mémoire interne (les poids du modèle), les systèmes RAG récupèrent d'abord des documents pertinents, puis les utilisent pour générer une réponse."),
        ("body", "Un pipeline RAG typique comprend : (1) l'indexation des documents dans une base vectorielle, (2) la transformation de la question en embedding, (3) la recherche des chunks les plus similaires, (4) l'augmentation du prompt avec ces chunks, et (5) la génération de la réponse par le LLM."),
        ("body", "ChromaDB est une base de données vectorielle open-source légère, idéale pour les projets locaux et le prototypage. Elle supporte les embeddings de différentes dimensions et offre des fonctionnalités de filtrage par métadonnées."),
        ("h2", "5. Les Applications de l'IA"),
        ("body", "L'IA est utilisée dans de nombreux domaines : la santé (diagnostic médical, découverte de médicaments), la finance (détection de fraude, trading algorithmique), l'éducation (tuteurs personnalisés, correction automatique), et les transports (véhicules autonomes)."),
        ("body", "Dans la data science, l'IA aide à automatiser l'exploration des données, à construire des modèles prédictifs plus précis, et à extraire des insights de grandes quantités de données non structurées comme les textes, images et vidéos."),
        ("body", "Les assistants IA comme ChatGPT, Copilot et Gemini transforment la façon dont nous travaillons. Ils peuvent rédiger des emails, générer du code, expliquer des concepts complexes, et même aider à la créativité artistique."),
        ("h2", "6. Éthique et Défis de l'IA"),
        ("body", "L'IA soulève des questions éthiques importantes : biais algorithmiques (les modèles peuvent reproduire ou amplifier des biais présents dans les données d'entraînement), confidentialité des données, transparence des décisions automatisées, et impact sur l'emploi."),
        ("body", "La réglementation de l'IA est en cours de développement dans de nombreux pays. L'Union Européenne a adopté l'AI Act, qui classe les systèmes d'IA selon leur niveau de risque et impose des obligations proportionnelles."),
        ("body", "La sécurité de l'IA (AI Safety) est un domaine de recherche qui vise à s'assurer que les systèmes d'IA agissent conformément aux intentions humaines, même lorsqu'ils deviennent très puissants. Les concepts d'alignement et de contrôle sont au cœur de ces recherches."),
    ]

    for style, text in content:
        if style == "h1":
            story.append(Paragraph(text, styles['Title']))
        elif style == "h2":
            story.append(Spacer(1, 0.5*cm))
            story.append(Paragraph(text, styles['Heading2']))
        else:
            story.append(Paragraph(text, styles['Normal']))
            story.append(Spacer(1, 0.3*cm))

    doc.build(story)
    print(f"✅ PDF de démonstration créé : {PDF_PATH}")
else:
    print(f"✅ PDF trouvé : {PDF_PATH}")

---
# 🚀 Section 6 — RAG Sans LangChain

> Cette implémentation utilise directement : **PyMuPDF** (extraction PDF) + **OpenAI API** (embeddings + génération) + **ChromaDB** (vector store)

---

## 6.1 Imports et Configuration

In [ ]:
import fitz  # PyMuPDF
import chromadb
import tiktoken
import time
import json
import re
from typing import List, Dict, Any, Optional, Tuple
from openai import OpenAI

# Configuration globale
EMBEDDING_MODEL = "text-embedding-3-small"  # Modèle d'embeddings OpenAI
LLM_MODEL = "gpt-4o-mini"                  # Modèle de génération
CHUNK_SIZE = 500                             # Taille des chunks en caractères
CHUNK_OVERLAP = 100                          # Chevauchement entre chunks
TOP_K = 3                                    # Nombre de chunks à récupérer
CHROMA_COLLECTION = "rag_demo"              # Nom de la collection ChromaDB

print("✅ Imports réussis")
print(f"📊 Configuration : chunk_size={CHUNK_SIZE}, overlap={CHUNK_OVERLAP}, top_k={TOP_K}")

## 6.2 Étape 1 : Extraction du texte depuis le PDF

In [ ]:
def extract_text_from_pdf(pdf_path: str) -> List[Dict[str, Any]]:
    """
    Extrait le texte d'un PDF page par page avec PyMuPDF.
    
    Args:
        pdf_path: Chemin vers le fichier PDF
        
    Returns:
        Liste de dicts {page_num, text, num_chars}
    """
    pages = []
    
    with fitz.open(pdf_path) as doc:
        print(f"📄 PDF ouvert : {doc.page_count} pages")
        
        for page_num, page in enumerate(doc, start=1):
            text = page.get_text()
            # Nettoyer le texte
            text = re.sub(r'\s+', ' ', text).strip()
            
            if text:  # Ignorer les pages vides
                pages.append({
                    "page_num": page_num,
                    "text": text,
                    "num_chars": len(text)
                })
    
    total_chars = sum(p["num_chars"] for p in pages)
    print(f"✅ Extraction terminée : {len(pages)} pages, {total_chars:,} caractères au total")
    return pages


# Extraire le texte du PDF
pages = extract_text_from_pdf(PDF_PATH)

# Afficher un aperçu
print("\n📝 Aperçu de la première page :")
print("-" * 60)
print(pages[0]["text"][:300] + "..." if len(pages[0]["text"]) > 300 else pages[0]["text"])

## 6.3 Étape 2 : Découpage en Chunks (Text Splitting)

In [ ]:
def split_text_into_chunks(
    pages: List[Dict[str, Any]],
    chunk_size: int = CHUNK_SIZE,
    chunk_overlap: int = CHUNK_OVERLAP
) -> List[Dict[str, Any]]:
    """
    Découpe le texte en chunks avec chevauchement.
    Stratégie : découpage récursif (d'abord par \n\n, puis par \n, puis par taille fixe).
    
    Args:
        pages: Liste des pages extraites du PDF
        chunk_size: Taille maximale d'un chunk en caractères
        chunk_overlap: Chevauchement entre chunks consécutifs
        
    Returns:
        Liste de chunks avec métadonnées
    """
    chunks = []
    chunk_id = 0
    
    for page in pages:
        text = page["text"]
        page_num = page["page_num"]
        
        # Découper d'abord par double saut de ligne (paragraphes)
        paragraphs = text.split("  ")  # Double espace après nettoyage
        if len(paragraphs) == 1:
            # Pas de paragraphes, découper par taille fixe avec overlap
            start = 0
            while start < len(text):
                end = min(start + chunk_size, len(text))
                chunk_text = text[start:end].strip()
                
                if len(chunk_text) > 50:  # Ignorer les chunks trop petits
                    chunks.append({
                        "id": f"chunk_{chunk_id}",
                        "text": chunk_text,
                        "page_num": page_num,
                        "start_char": start,
                        "num_chars": len(chunk_text)
                    })
                    chunk_id += 1
                
                # Avancer en tenant compte de l'overlap
                start += chunk_size - chunk_overlap
        else:
            # Regrouper les paragraphes en chunks
            current_chunk = ""
            start_char = 0
            
            for para in paragraphs:
                para = para.strip()
                if not para:
                    continue
                
                if len(current_chunk) + len(para) + 1 <= chunk_size:
                    current_chunk += (" " if current_chunk else "") + para
                else:
                    if current_chunk:
                        chunks.append({
                            "id": f"chunk_{chunk_id}",
                            "text": current_chunk,
                            "page_num": page_num,
                            "start_char": start_char,
                            "num_chars": len(current_chunk)
                        })
                        chunk_id += 1
                        # Overlap: reprendre depuis la fin du dernier chunk
                        overlap_text = current_chunk[-chunk_overlap:] if len(current_chunk) > chunk_overlap else current_chunk
                        current_chunk = overlap_text + " " + para
                    else:
                        current_chunk = para
                    start_char = len(text) - len(para)
            
            if current_chunk:  # Dernier chunk
                chunks.append({
                    "id": f"chunk_{chunk_id}",
                    "text": current_chunk,
                    "page_num": page_num,
                    "start_char": start_char,
                    "num_chars": len(current_chunk)
                })
                chunk_id += 1
    
    print(f"✅ {len(chunks)} chunks créés")
    print(f"📏 Taille moyenne d'un chunk : {sum(c['num_chars'] for c in chunks) // len(chunks)} caractères")
    return chunks


# Découper en chunks
chunks = split_text_into_chunks(pages)

# Afficher quelques chunks
print("\n📦 Exemples de chunks :")
for i, chunk in enumerate(chunks[:3]):
    print(f"\n--- Chunk {i+1} (page {chunk['page_num']}, {chunk['num_chars']} chars) ---")
    print(chunk["text"][:200] + "..." if len(chunk["text"]) > 200 else chunk["text"])

## 6.4 Étape 3 : Création des Embeddings et Indexation dans ChromaDB

In [ ]:
def setup_chroma_and_index(
    chunks: List[Dict[str, Any]],
    collection_name: str = CHROMA_COLLECTION,
    embedding_model: str = EMBEDDING_MODEL
) -> Tuple[chromadb.Collection, OpenAI]:
    """
    Crée les embeddings via OpenAI et les indexe dans ChromaDB.
    
    Args:
        chunks: Liste des chunks à indexer
        collection_name: Nom de la collection ChromaDB
        embedding_model: Modèle d'embeddings OpenAI
        
    Returns:
        (collection ChromaDB, client OpenAI)
    """
    # Initialiser le client OpenAI
    client = OpenAI(api_key=OPENAI_API_KEY)
    
    # Initialiser ChromaDB (mode persistant local)
    chroma_client = chromadb.PersistentClient(path="./chroma_db")
    
    # Supprimer la collection si elle existe déjà (pour repartir propre)
    try:
        chroma_client.delete_collection(collection_name)
        print(f"🗑️ Collection existante '{collection_name}' supprimée")
    except Exception:
        pass
    
    # Créer une nouvelle collection
    collection = chroma_client.create_collection(
        name=collection_name,
        metadata={"hnsw:space": "cosine"}  # Métrique de similarité cosinus
    )
    
    print(f"\n🔢 Génération des embeddings pour {len(chunks)} chunks...")
    print(f"   Modèle : {embedding_model}")
    
    # Générer les embeddings par lots (batch) pour optimiser les appels API
    batch_size = 20
    all_ids = []
    all_embeddings = []
    all_documents = []
    all_metadatas = []
    
    for i in range(0, len(chunks), batch_size):
        batch = chunks[i:i + batch_size]
        texts = [c["text"] for c in batch]
        
        # Appel API OpenAI pour les embeddings
        response = client.embeddings.create(
            input=texts,
            model=embedding_model
        )
        
        # Collecter les résultats
        for j, chunk in enumerate(batch):
            all_ids.append(chunk["id"])
            all_embeddings.append(response.data[j].embedding)
            all_documents.append(chunk["text"])
            all_metadatas.append({
                "page_num": chunk["page_num"],
                "num_chars": chunk["num_chars"],
                "start_char": chunk["start_char"]
            })
        
        print(f"   Lot {i//batch_size + 1}/{(len(chunks)-1)//batch_size + 1} traité ({min(i+batch_size, len(chunks))}/{len(chunks)} chunks)")
    
    # Insérer tous les embeddings dans ChromaDB
    collection.add(
        ids=all_ids,
        embeddings=all_embeddings,
        documents=all_documents,
        metadatas=all_metadatas
    )
    
    print(f"\n✅ Indexation terminée !")
    print(f"   📊 {collection.count()} chunks indexés dans ChromaDB")
    print(f"   💾 Base vectorielle sauvegardée dans './chroma_db'")
    
    return collection, client


# Indexer les chunks (nécessite une clé API OpenAI valide)
if OPENAI_API_KEY:
    collection, openai_client = setup_chroma_and_index(chunks)
else:
    print("⚠️ Clé API OpenAI manquante - indexation simulée")
    print("   Pour une démo sans clé API, voir la section alternative ci-dessous")

In [ ]:
# Alternative sans clé API : utiliser des embeddings locaux (sentence-transformers)
# Décommentez cette cellule si vous n'avez pas de clé OpenAI

# !pip install -q sentence-transformers

# from sentence_transformers import SentenceTransformer

# def setup_chroma_local(chunks, collection_name="rag_demo_local"):
#     """Version locale sans API OpenAI, utilise sentence-transformers"""
#     model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")
#     
#     chroma_client = chromadb.PersistentClient(path="./chroma_db_local")
#     try:
#         chroma_client.delete_collection(collection_name)
#     except Exception:
#         pass
#     
#     collection = chroma_client.create_collection(
#         name=collection_name,
#         metadata={"hnsw:space": "cosine"}
#     )
#     
#     texts = [c["text"] for c in chunks]
#     print("Génération des embeddings locaux...")
#     embeddings = model.encode(texts, show_progress_bar=True).tolist()
#     
#     collection.add(
#         ids=[c["id"] for c in chunks],
#         embeddings=embeddings,
#         documents=texts,
#         metadatas=[{"page_num": c["page_num"]} for c in chunks]
#     )
#     print(f"✅ {collection.count()} chunks indexés (local)")
#     return collection, model
#
# collection_local, st_model = setup_chroma_local(chunks)
print("💡 Décommentez la cellule ci-dessus pour une version sans clé API (sentence-transformers)")

## 6.5 Étape 4 : Retrieval (Récupération des chunks pertinents)

In [ ]:
def retrieve_relevant_chunks(
    question: str,
    collection: chromadb.Collection,
    client: OpenAI,
    top_k: int = TOP_K,
    embedding_model: str = EMBEDDING_MODEL
) -> List[Dict[str, Any]]:
    """
    Récupère les chunks les plus pertinents pour une question.
    
    Args:
        question: La question de l'utilisateur
        collection: Collection ChromaDB
        client: Client OpenAI
        top_k: Nombre de chunks à récupérer
        embedding_model: Modèle d'embeddings
        
    Returns:
        Liste des chunks récupérés avec leurs scores de similarité
    """
    start_time = time.time()
    
    # Créer l'embedding de la question
    query_response = client.embeddings.create(
        input=[question],
        model=embedding_model
    )
    query_embedding = query_response.data[0].embedding
    
    # Rechercher dans ChromaDB
    results = collection.query(
        query_embeddings=[query_embedding],
        n_results=top_k,
        include=["documents", "metadatas", "distances"]
    )
    
    retrieval_time = (time.time() - start_time) * 1000
    
    # Formater les résultats
    retrieved = []
    for i in range(len(results["ids"][0])):
        # ChromaDB retourne la distance cosinus (0 = identique, 2 = opposé)
        # On convertit en score de similarité (0 à 1)
        distance = results["distances"][0][i]
        similarity = 1 - (distance / 2)  # Normalisation pour cosinus
        
        retrieved.append({
            "rank": i + 1,
            "id": results["ids"][0][i],
            "text": results["documents"][0][i],
            "metadata": results["metadatas"][0][i],
            "distance": distance,
            "similarity": round(similarity, 4)
        })
    
    print(f"⚡ Retrieval en {retrieval_time:.1f}ms")
    return retrieved


# Test du retrieval
if OPENAI_API_KEY:
    test_question = "Qu'est-ce que le machine learning ?"
    print(f"🔍 Question : {test_question}")
    print("-" * 60)
    
    retrieved_chunks = retrieve_relevant_chunks(test_question, collection, openai_client)
    
    print(f"\n📋 Top-{TOP_K} chunks récupérés :")
    for chunk in retrieved_chunks:
        print(f"\n🏆 Rang {chunk['rank']} | Similarité: {chunk['similarity']} | Page: {chunk['metadata']['page_num']}")
        print(f"   {chunk['text'][:200]}...")
else:
    print("⚠️ Clé API OpenAI manquante")

## 6.6 Étape 5 : Génération de la Réponse avec le LLM

In [ ]:
def build_prompt(question: str, retrieved_chunks: List[Dict[str, Any]]) -> str:
    """
    Construit le prompt enrichi avec le contexte récupéré.
    
    Args:
        question: La question de l'utilisateur
        retrieved_chunks: Les chunks récupérés par le retriever
        
    Returns:
        Prompt formaté prêt pour le LLM
    """
    # Construire le contexte
    context_parts = []
    for chunk in retrieved_chunks:
        context_parts.append(
            f"[Source - Page {chunk['metadata']['page_num']}]\n{chunk['text']}"
        )
    
    context = "\n\n---\n\n".join(context_parts)
    
    prompt = f"""Contexte extrait du document :

{context}

---

Question : {question}

Instructions :
- Réponds à la question en te basant UNIQUEMENT sur le contexte fourni ci-dessus.
- Si le contexte ne contient pas l'information nécessaire, dis-le clairement.
- Sois précis, structuré et cite les sources (numéros de page) si pertinent.
- Réponds en français.

Réponse :"""
    
    return prompt


def generate_answer(
    question: str,
    retrieved_chunks: List[Dict[str, Any]],
    client: OpenAI,
    model: str = LLM_MODEL,
    temperature: float = 0.2,
    max_tokens: int = 800
) -> Dict[str, Any]:
    """
    Génère une réponse avec le LLM basée sur les chunks récupérés.
    
    Args:
        question: La question de l'utilisateur
        retrieved_chunks: Les chunks récupérés
        client: Client OpenAI
        model: Modèle LLM à utiliser
        temperature: Température (0=déterministe, 1=créatif)
        max_tokens: Nombre max de tokens à générer
        
    Returns:
        Dict avec la réponse, le modèle utilisé et les métriques
    """
    start_time = time.time()
    
    # Construire le prompt
    user_prompt = build_prompt(question, retrieved_chunks)
    
    # Appel au LLM
    response = client.chat.completions.create(
        model=model,
        messages=[
            {
                "role": "system",
                "content": "Tu es un assistant expert qui répond aux questions en se basant exclusivement sur les documents fournis. Tu es précis, structuré et toujours honnête quand une information n'est pas disponible dans le contexte."
            },
            {
                "role": "user",
                "content": user_prompt
            }
        ],
        temperature=temperature,
        max_tokens=max_tokens
    )
    
    generation_time = (time.time() - start_time) * 1000
    
    return {
        "answer": response.choices[0].message.content,
        "model": model,
        "generation_time_ms": round(generation_time, 1),
        "prompt_tokens": response.usage.prompt_tokens,
        "completion_tokens": response.usage.completion_tokens,
        "total_tokens": response.usage.total_tokens
    }


# Test de la génération
if OPENAI_API_KEY:
    print(f"🤖 Génération de réponse avec {LLM_MODEL}...")
    print("-" * 60)
    
    generation_result = generate_answer(test_question, retrieved_chunks, openai_client)
    
    print(f"✅ Réponse générée en {generation_result['generation_time_ms']}ms")
    print(f"📊 Tokens utilisés : {generation_result['total_tokens']} (prompt: {generation_result['prompt_tokens']}, réponse: {generation_result['completion_tokens']})")
    print("\n" + "=" * 60)
    print("📝 RÉPONSE :")
    print("=" * 60)
    print(generation_result["answer"])
else:
    print("⚠️ Clé API OpenAI manquante")

## 6.7 Pipeline RAG Complet — Classe SimpleRAG

In [ ]:
class SimpleRAG:
    """
    Pipeline RAG complet sans LangChain.
    Utilise : PyMuPDF + OpenAI Embeddings + ChromaDB + OpenAI GPT
    """
    
    def __init__(
        self,
        openai_api_key: str,
        embedding_model: str = "text-embedding-3-small",
        llm_model: str = "gpt-4o-mini",
        chunk_size: int = 500,
        chunk_overlap: int = 100,
        top_k: int = 3,
        chroma_path: str = "./chroma_db",
        collection_name: str = "simple_rag"
    ):
        self.client = OpenAI(api_key=openai_api_key)
        self.embedding_model = embedding_model
        self.llm_model = llm_model
        self.chunk_size = chunk_size
        self.chunk_overlap = chunk_overlap
        self.top_k = top_k
        self.collection_name = collection_name
        
        # Initialiser ChromaDB
        self.chroma_client = chromadb.PersistentClient(path=chroma_path)
        self.collection = None
        
        # Mémoire conversationnelle (fenêtre des N derniers échanges)
        self.conversation_history = []
        self.memory_window = 5  # Garder les 5 derniers échanges
        
        print(f"✅ SimpleRAG initialisé")
        print(f"   Embedding: {embedding_model} | LLM: {llm_model}")
        print(f"   Chunk size: {chunk_size} | Overlap: {chunk_overlap} | Top-K: {top_k}")
    
    def index_pdf(self, pdf_path: str) -> int:
        """Indexe un PDF dans ChromaDB. Retourne le nombre de chunks."""
        print(f"\n📄 Indexation de : {pdf_path}")
        
        # Extraire le texte
        pages = extract_text_from_pdf(pdf_path)
        
        # Découper en chunks
        chunks = split_text_into_chunks(pages, self.chunk_size, self.chunk_overlap)
        
        # Créer/recréer la collection
        try:
            self.chroma_client.delete_collection(self.collection_name)
        except Exception:
            pass
        self.collection = self.chroma_client.create_collection(
            name=self.collection_name,
            metadata={"hnsw:space": "cosine"}
        )
        
        # Générer et indexer les embeddings par lots
        batch_size = 20
        for i in range(0, len(chunks), batch_size):
            batch = chunks[i:i + batch_size]
            texts = [c["text"] for c in batch]
            
            response = self.client.embeddings.create(
                input=texts, model=self.embedding_model
            )
            
            self.collection.add(
                ids=[c["id"] for c in batch],
                embeddings=[r.embedding for r in response.data],
                documents=texts,
                metadatas=[{"page_num": c["page_num"], "num_chars": c["num_chars"]} for c in batch]
            )
        
        print(f"✅ {self.collection.count()} chunks indexés")
        return self.collection.count()
    
    def query(
        self,
        question: str,
        use_memory: bool = True
    ) -> Dict[str, Any]:
        """
        Pipeline RAG complet : retrieve + generate.
        Supporte la mémoire conversationnelle.
        
        Args:
            question: La question de l'utilisateur
            use_memory: Si True, inclut l'historique de conversation dans le prompt
            
        Returns:
            Dict avec la réponse, les sources et les métriques
        """
        if self.collection is None:
            raise ValueError("Aucun document indexé. Appelez d'abord index_pdf().")
        
        start_total = time.time()
        
        # 1. RETRIEVAL : Trouver les chunks pertinents
        t_retrieve = time.time()
        query_embedding = self.client.embeddings.create(
            input=[question], model=self.embedding_model
        ).data[0].embedding
        
        results = self.collection.query(
            query_embeddings=[query_embedding],
            n_results=self.top_k,
            include=["documents", "metadatas", "distances"]
        )
        retrieval_time = (time.time() - t_retrieve) * 1000
        
        retrieved_chunks = [
            {
                "text": results["documents"][0][i],
                "metadata": results["metadatas"][0][i],
                "similarity": round(1 - results["distances"][0][i] / 2, 4)
            }
            for i in range(len(results["ids"][0]))
        ]
        
        # 2. AUGMENTATION : Construire le prompt enrichi
        context = "\n\n---\n\n".join(
            f"[Page {c['metadata']['page_num']}]\n{c['text']}"
            for c in retrieved_chunks
        )
        
        # 3. GENERATION : Appel au LLM
        t_generate = time.time()
        
        messages = [
            {
                "role": "system",
                "content": "Tu es un assistant expert. Réponds uniquement en te basant sur le contexte fourni. Si l'information n'est pas dans le contexte, dis-le clairement."
            }
        ]
        
        # Ajouter l'historique de conversation (mémoire window)
        if use_memory and self.conversation_history:
            recent_history = self.conversation_history[-self.memory_window:]
            messages.extend(recent_history)
        
        # Ajouter la question avec contexte
        messages.append({
            "role": "user",
            "content": f"Contexte :\n{context}\n\nQuestion : {question}\n\nRéponds en français."
        })
        
        llm_response = self.client.chat.completions.create(
            model=self.llm_model,
            messages=messages,
            temperature=0.2,
            max_tokens=800
        )
        
        answer = llm_response.choices[0].message.content
        generation_time = (time.time() - t_generate) * 1000
        total_time = (time.time() - start_total) * 1000
        
        # Mettre à jour la mémoire conversationnelle
        self.conversation_history.append({"role": "user", "content": question})
        self.conversation_history.append({"role": "assistant", "content": answer})
        
        return {
            "question": question,
            "answer": answer,
            "sources": [
                {"page": c["metadata"]["page_num"], "similarity": c["similarity"], "excerpt": c["text"][:150] + "..."}
                for c in retrieved_chunks
            ],
            "metrics": {
                "retrieval_ms": round(retrieval_time, 1),
                "generation_ms": round(generation_time, 1),
                "total_ms": round(total_time, 1),
                "tokens_used": llm_response.usage.total_tokens,
                "model": self.llm_model
            }
        }
    
    def clear_memory(self):
        """Efface l'historique de conversation."""
        self.conversation_history = []
        print("🧹 Mémoire conversationnelle effacée")


print("✅ Classe SimpleRAG définie")

In [ ]:
# Démonstration du SimpleRAG
if OPENAI_API_KEY:
    # Créer et indexer
    rag = SimpleRAG(
        openai_api_key=OPENAI_API_KEY,
        collection_name="simple_rag_demo"
    )
    rag.index_pdf(PDF_PATH)
    
    print("\n" + "=" * 60)
    print("🎯 TEST DU PIPELINE RAG COMPLET")
    print("=" * 60)
    
    # Questions de test
    questions = [
        "Qu'est-ce que le deep learning et comment fonctionne-t-il ?",
        "Quels sont les défis éthiques de l'IA ?",
        "Comment fonctionne le RAG ?"
    ]
    
    for q in questions:
        print(f"\n❓ Question : {q}")
        print("-" * 50)
        result = rag.query(q)
        print(f"💬 Réponse : {result['answer']}")
        print(f"\n📚 Sources utilisées :")
        for src in result["sources"]:
            print(f"   • Page {src['page']} (similarité: {src['similarity']}) : {src['excerpt'][:100]}...")
        print(f"\n⏱️ Métriques : retrieval={result['metrics']['retrieval_ms']}ms | génération={result['metrics']['generation_ms']}ms | tokens={result['metrics']['tokens_used']}")
        print("=" * 60)
else:
    print("⚠️ Clé API OpenAI manquante - démonstration non disponible")

In [ ]:
# Test de la mémoire conversationnelle
if OPENAI_API_KEY:
    print("💭 TEST DE LA MÉMOIRE CONVERSATIONNELLE")
    print("=" * 60)
    
    rag.clear_memory()
    
    # Conversation en plusieurs tours
    conversation = [
        "Qu'est-ce que le machine learning ?",
        "Et quels en sont les différents types ?",  # 'en' réfère au ML → test de la mémoire
        "Donne-moi un exemple concret d'application."
    ]
    
    for turn, question in enumerate(conversation, 1):
        print(f"\n[Tour {turn}] 👤 User : {question}")
        result = rag.query(question, use_memory=True)
        print(f"[Tour {turn}] 🤖 Bot  : {result['answer'][:300]}...")
    
    print(f"\n📝 Historique en mémoire : {len(rag.conversation_history)} messages")
else:
    print("⚠️ Clé API OpenAI manquante")

---
# 🦜 Section 7 — RAG Avec LangChain

> La même fonctionnalité, mais en utilisant les abstractions de LangChain pour un code plus concis et plus facilement extensible.

---

## 7.1 Pourquoi LangChain ?

| Aspect | Sans LangChain | Avec LangChain |
|--------|---------------|----------------|
| **Code** | Plus verbeux, mais explicite | Concis, abstrait |
| **Flexibilité** | Contrôle total | Parfois contraignant |
| **Écosystème** | À construire | 100+ intégrations prêtes |
| **Debugging** | Facile | Plus difficile (abstractions) |
| **Production** | Adapté | Adapté + outils LangSmith |
| **Apprentissage** | Comprendre le fond | Productivité rapide |

In [ ]:
# Imports LangChain
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
import textwrap

print("✅ Imports LangChain réussis")

## 7.2 Chargement et Indexation avec LangChain

In [ ]:
if OPENAI_API_KEY:
    # ── 1. Charger le PDF ──────────────────────────────────────────
    loader = PyMuPDFLoader(PDF_PATH)
    documents = loader.load()
    print(f"📄 {len(documents)} pages chargées avec LangChain")
    
    # ── 2. Découper en chunks ──────────────────────────────────────
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=100,
        length_function=len,
        separators=["\n\n", "\n", ". ", " ", ""]
    )
    lc_chunks = text_splitter.split_documents(documents)
    print(f"✅ {len(lc_chunks)} chunks créés avec RecursiveCharacterTextSplitter")
    print(f"📏 Taille moyenne : {sum(len(c.page_content) for c in lc_chunks) // len(lc_chunks)} caractères")
    
    # Aperçu d'un chunk
    print(f"\n🔍 Exemple de chunk :")
    print(f"   Métadonnées : {lc_chunks[0].metadata}")
    print(f"   Contenu     : {lc_chunks[0].page_content[:200]}...")
    
    # ── 3. Créer les embeddings et indexer dans ChromaDB ──────────
    embeddings = OpenAIEmbeddings(
        model="text-embedding-3-small",
        api_key=OPENAI_API_KEY
    )
    
    # Créer le vector store ChromaDB
    lc_vectorstore = Chroma.from_documents(
        documents=lc_chunks,
        embedding=embeddings,
        collection_name="langchain_rag",
        persist_directory="./chroma_db_langchain"
    )
    
    print(f"\n✅ Vector store LangChain créé")
    print(f"   📊 {lc_vectorstore._collection.count()} chunks dans ChromaDB")
else:
    print("⚠️ Clé API OpenAI manquante")

## 7.3 Pipeline RAG Simple avec LangChain (LCEL)

In [ ]:
if OPENAI_API_KEY:
    # ── Créer le retriever ────────────────────────────────────────
    retriever = lc_vectorstore.as_retriever(
        search_type="similarity",
        search_kwargs={"k": 3}
    )
    
    # ── Créer le LLM ─────────────────────────────────────────────
    llm = ChatOpenAI(
        model="gpt-4o-mini",
        temperature=0.2,
        api_key=OPENAI_API_KEY
    )
    
    # ── Définir le prompt template ────────────────────────────────
    RAG_PROMPT = ChatPromptTemplate.from_template("""
Tu es un assistant expert. Réponds à la question en te basant UNIQUEMENT sur le contexte fourni.
Si l'information n'est pas dans le contexte, dis-le clairement.
Réponds en français.

Contexte :
{context}

Question : {question}

Réponse :
""")
    
    # ── Fonction pour formater les documents récupérés ────────────
    def format_docs(docs):
        return "\n\n".join(
            f"[Page {doc.metadata.get('page', '?')+1}]\n{doc.page_content}"
            for doc in docs
        )
    
    # ── Construire la chaîne RAG avec LCEL ────────────────────────
    # LCEL = LangChain Expression Language (syntaxe | pour chaîner)
    rag_chain = (
        {
            "context": retriever | format_docs,  # Retriever → formatage
            "question": RunnablePassthrough()     # Question passée directement
        }
        | RAG_PROMPT      # Injection dans le prompt template
        | llm              # Appel au LLM
        | StrOutputParser() # Extraction du texte de réponse
    )
    
    print("✅ Pipeline RAG LangChain créé avec LCEL")
    print("   Chaîne : retriever → format_docs → prompt → llm → parser")
else:
    print("⚠️ Clé API OpenAI manquante")

In [ ]:
# Test du pipeline LangChain
if OPENAI_API_KEY:
    test_questions_lc = [
        "Qu'est-ce que le deep learning ?",
        "Quels modèles LLM sont mentionnés dans le document ?"
    ]
    
    print("🦜 TEST DU RAG LANGCHAIN")
    print("=" * 60)
    
    for question in test_questions_lc:
        print(f"\n❓ Question : {question}")
        start = time.time()
        answer = rag_chain.invoke(question)
        elapsed = (time.time() - start) * 1000
        print(f"💬 Réponse ({elapsed:.0f}ms) :")
        print(answer)
        print("-" * 60)
else:
    print("⚠️ Clé API OpenAI manquante")

## 7.4 RAG Conversationnel avec LangChain (avec mémoire)

In [ ]:
if OPENAI_API_KEY:
    # Prompt pour reformuler la question en tenant compte de l'historique
    CONTEXTUALIZE_Q_PROMPT = ChatPromptTemplate.from_messages([
        ("system",
         "Étant donné l'historique de conversation et la dernière question de l'utilisateur "
         "qui peut référencer le contexte de la conversation, reformule la question pour qu'elle "
         "soit autonome et compréhensible sans l'historique. "
         "Ne réponds PAS à la question, reformule-la seulement si nécessaire, sinon retourne-la telle quelle."
        ),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}")
    ])
    
    # Prompt pour la réponse finale
    QA_PROMPT = ChatPromptTemplate.from_messages([
        ("system",
         "Tu es un assistant expert. Réponds à la question en te basant UNIQUEMENT sur le contexte fourni. "
         "Si l'information n'est pas disponible, dis-le clairement. Réponds en français.\n\n"
         "Contexte :\n{context}"
        ),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}")
    ])
    
    # Chaîne de reformulation de question
    contextualize_q_chain = CONTEXTUALIZE_Q_PROMPT | llm | StrOutputParser()
    
    def contextualized_question(inputs: dict):
        """Reformule la question si un historique est présent."""
        if inputs.get("chat_history"):
            return contextualize_q_chain
        return inputs["input"]
    
    # Pipeline RAG conversationnel
    conversational_rag_chain = (
        RunnablePassthrough.assign(
            context=contextualized_question | retriever | format_docs
        )
        | QA_PROMPT
        | llm
        | StrOutputParser()
    )
    
    print("✅ RAG Conversationnel LangChain créé")
    
    # Test de conversation multi-tours
    print("\n💭 TEST DE CONVERSATION MULTI-TOURS")
    print("=" * 60)
    
    chat_history = []
    
    conv_questions = [
        "Qu'est-ce que le machine learning ?",
        "Quels en sont les différents types ?",
        "Donne un exemple d'application réelle."
    ]
    
    for turn, question in enumerate(conv_questions, 1):
        print(f"\n[Tour {turn}] 👤 User : {question}")
        
        answer = conversational_rag_chain.invoke({
            "input": question,
            "chat_history": chat_history
        })
        
        # Mettre à jour l'historique
        chat_history.extend([
            HumanMessage(content=question),
            AIMessage(content=answer)
        ])
        
        print(f"[Tour {turn}] 🤖 Bot  : {answer[:350]}..." if len(answer) > 350 else f"[Tour {turn}] 🤖 Bot  : {answer}")
        print(f"            💾 Historique : {len(chat_history)} messages")
else:
    print("⚠️ Clé API OpenAI manquante")

## 7.5 RAG Avancé avec LangChain : Multi-Query Retriever

In [ ]:
if OPENAI_API_KEY:
    from langchain.retrievers.multi_query import MultiQueryRetriever
    
    # Multi-Query Retriever : génère plusieurs variantes de la question
    # pour maximiser le recall
    multi_query_retriever = MultiQueryRetriever.from_llm(
        retriever=lc_vectorstore.as_retriever(search_kwargs={"k": 3}),
        llm=llm
    )
    
    # Activer les logs pour voir les questions générées
    import logging
    logging.basicConfig()
    logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)
    
    test_q = "Comment l'IA est-elle utilisée dans la santé ?"
    print(f"🔍 Multi-Query Retriever — Question originale : {test_q}")
    print("-" * 60)
    
    # Récupérer avec plusieurs requêtes générées automatiquement
    multi_docs = multi_query_retriever.invoke(test_q)
    
    print(f"\n📋 {len(multi_docs)} chunks récupérés (dédupliqués) :")
    for i, doc in enumerate(multi_docs[:3], 1):
        print(f"\n  [{i}] Page {doc.metadata.get('page', '?')+1} : {doc.page_content[:150]}...")
else:
    print("⚠️ Clé API OpenAI manquante")

---
# ⚖️ Section 8 — Comparaison et Bonnes Pratiques

---

## 8.1 Sans LangChain vs Avec LangChain

In [ ]:
# Comparaison visuelle du code pour la même opération

print("\n📊 COMPARAISON DE CODE")
print("=" * 70)

print("""
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
SANS LANGCHAIN (code explicite, ~10 lignes pour une query)
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

  # 1. Embed la question
  embedding = client.embeddings.create(input=[q], model="text-embedding-3-small")
  
  # 2. Chercher dans ChromaDB
  results = collection.query(query_embeddings=[embedding.data[0].embedding], n_results=3)
  
  # 3. Construire le contexte
  context = "\\n\\n".join(results["documents"][0])
  
  # 4. Appeler le LLM
  response = client.chat.completions.create(
      model="gpt-4o-mini",
      messages=[{"role": "user", "content": f"Contexte: {context}\\n\\nQuestion: {q}"}]
  )
  answer = response.choices[0].message.content

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
AVEC LANGCHAIN LCEL (abstrait, ~3 lignes pour une query)
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

  rag_chain = (
      {"context": retriever | format_docs, "question": RunnablePassthrough()}
      | prompt | llm | StrOutputParser()
  )
  answer = rag_chain.invoke(question)

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
""")

## 8.2 Guide de Décision : Quand Utiliser Quoi ?

```
Votre situation
      │
      ├─ Prototypage rapide / apprentissage ?  →  LangChain (productivité)
      │
      ├─ Comprendre le fonctionnement profond ? →  Sans LangChain (transparence)
      │
      ├─ Contrôle total du pipeline ?           →  Sans LangChain
      │
      ├─ Intégrer 10+ sources de données ?      →  LangChain (connecteurs)
      │
      ├─ Besoin d'agents/outils complexes ?     →  LangGraph (sur LangChain)
      │
      └─ Production avec monitoring ?           →  LangSmith + LangChain
```

## 8.3 Bonnes Pratiques RAG

### 🔧 Optimisation du Chunking
```
Type de document       Chunk size recommandé    Overlap
──────────────         ────────────────────     ───────
FAQ                    200–300 tokens           20%
Articles/Blog          400–600 tokens           15%
Documentation tech.    300–500 tokens           20%
Contrats/Legal         500–800 tokens           10%
Code source            200–400 tokens           10%
```

### 🎯 Optimisation du Retrieval
- **Top-K** : Commencer avec k=3-5, augmenter si les réponses manquent d'info
- **Hybrid search** : Combiner dense (sémantique) + sparse (BM25) pour plus de robustesse
- **Reranking** : Utiliser un cross-encoder après le retrieval initial
- **Metadata filtering** : Filtrer par date, catégorie, source pour préciser la recherche

### 💡 Optimisation du Prompt
- Instruire le LLM de citer les sources
- Demander explicitement de signaler l'absence d'information
- Utiliser temperature ≤ 0.3 pour les Q&A factuels
- Tester différents formats de contexte (liste, paragraphe, JSON)

### 📊 Évaluation du RAG

| Métrique | Description | Outil |
|----------|-------------|-------|
| **Faithfulness** | La réponse est-elle fondée sur le contexte ? | RAGAS |
| **Answer Relevancy** | La réponse répond-elle à la question ? | RAGAS |
| **Context Precision** | Les chunks récupérés sont-ils pertinents ? | RAGAS |
| **Context Recall** | Tous les chunks nécessaires ont-ils été récupérés ? | RAGAS |

In [ ]:
# Résumé récapitulatif
print("""
╔══════════════════════════════════════════════════════════════════════╗
║                    📚 RÉCAPITULATIF DU COURS RAG                    ║
╠══════════════════════════════════════════════════════════════════════╣
║                                                                      ║
║  RAG = Retrieval-Augmented Generation                                ║
║  Objectif : Ancrer les réponses LLM dans des faits vérifiables      ║
║                                                                      ║
║  PIPELINE :                                                          ║
║  PDF → Chunking → Embeddings → ChromaDB → Retrieval → LLM → Answer  ║
║                                                                      ║
║  TYPES DE RAG :                                                      ║
║  • Naive RAG     : Simple, efficace pour des Q&A basiques           ║
║  • Advanced RAG  : Query rewriting, reranking, HyDE                 ║
║  • Modular RAG   : Multi-sources, agents, Self-RAG, Graph RAG       ║
║                                                                      ║
║  MÉMOIRE :                                                           ║
║  • Short-term    : In-context (conversation history)                 ║
║  • Long-term     : Vector store (sémantique) / DB (structuré)       ║
║  • Stratégies    : Buffer, Window, Summary, Vector memory           ║
║                                                                      ║
║  IMPLÉMENTATIONS :                                                   ║
║  • Sans LangChain : Contrôle total, code explicite                  ║
║  • Avec LangChain : Concis, écosystème riche, LCEL                  ║
║                                                                      ║
╚══════════════════════════════════════════════════════════════════════╝
""")

---
## 🎓 Exercices Pratiques

### Exercice 1 : Tester avec votre propre PDF
1. Remplacer `PDF_PATH` par le chemin vers votre PDF
2. Re-exécuter les cellules d'indexation
3. Tester des questions spécifiques à votre document

### Exercice 2 : Implémenter le reranking
1. Installer `sentence-transformers`
2. Après le retrieval, utiliser un cross-encoder pour re-classer les chunks
3. Comparer les réponses avant/après reranking

### Exercice 3 : Hybrid Search
1. Implémenter une recherche BM25 avec `rank_bm25`
2. Combiner les scores BM25 et cosinus (Reciprocal Rank Fusion)
3. Mesurer l'amélioration du recall

### Exercice 4 : Évaluer le RAG avec RAGAS
1. Installer `ragas`
2. Créer un dataset de questions/réponses de référence
3. Mesurer faithfulness, answer_relevancy, context_precision

### Exercice 5 : RAG avec filtrage par métadonnées
1. Indexer plusieurs PDFs avec des métadonnées différentes (auteur, date, catégorie)
2. Permettre à l'utilisateur de filtrer par source
3. Tester `collection.query(where={"category": "IA"}, ...)` dans ChromaDB

---

## 📚 Ressources

- 📄 [RAG Paper original (Lewis et al., 2020)](https://arxiv.org/abs/2005.11401)
- 📄 [Survey on RAG (Gao et al., 2023)](https://arxiv.org/abs/2312.10997)
- 🌐 [ChromaDB Documentation](https://docs.trychroma.com/)
- 🌐 [LangChain RAG Guide](https://python.langchain.com/docs/tutorials/rag/)
- 🌐 [OpenAI Embeddings](https://platform.openai.com/docs/guides/embeddings)
- 🛠️ [RAGAS — Evaluation Framework](https://docs.ragas.io/)

---

**🎉 Félicitations ! Vous avez complété le cours RAG complet !**

Vous savez maintenant :
- ✅ Ce qu'est le RAG et ses composants
- ✅ Les différents types de RAG et quand les utiliser
- ✅ Gérer la mémoire dans un système RAG
- ✅ Implémenter un RAG de A à Z sans LangChain
- ✅ Utiliser LangChain pour simplifier le pipeline